In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [3]:
import numpy as np
from tqdm.auto import tqdm

from hcrot import layers, optim
from hcrot.dataset import *
from hcrot.utils import *

In [4]:
sentences = [
    "The sun always rises after the darkest night, so keep hope alive",
    "A bird in the hand is worth two in the bush, so hold on tightly",
    "Fortune favors the brave, but wisdom guides their steps",
    "The squeaky wheel gets the grease, so speak up when needed",
    "A watched pot never boils, so do not waste your time staring",
    "He who hesitates is lost, so act decisively when the time comes",
    "A penny saved is a penny earned, so start saving early",
    "A chain is only as strong as its weakest link, so strengthen every part",
    "Absence makes the heart grow fonder, but do not forget to stay in touch",
    "Do unto others as you would have them do unto you, and live with kindness",
    "The harder you work, the luckier you get, so never stop trying",
    "Patience is a virtue, but persistence brings success",
    "A rolling stone gathers no moss, so keep moving forward",
    "Great minds think alike, but they also think differently",
    "Birds of a feather flock together, so choose your company wisely",
    "A fool and his money are soon parted, so spend with caution",
    "Time heals all wounds, but some scars may remain",
    "A stitch in time saves nine, so do not delay your efforts",
    "Honesty is the best policy, but tact is also important",
    "If it is not broken, do not fix it, but always seek improvement",
    "Necessity is the mother of invention, so embrace challenges",
    "Do not bite the hand that feeds you, but show gratitude instead",
    "An ounce of prevention is worth a pound of cure, so plan ahead",
    "A little knowledge is a dangerous thing, so always keep learning",
    "Practice what you preach, but also be open to feedback",
    "Where there is smoke, there is fire, so investigate further",
    "Every dog has its day, so your time will come too",
    "Do not put all your eggs in one basket, but diversify your risks",
    "The best things in life are free, but they require effort",
    "A journey well planned is a journey half completed"
]

## Tokenize

In [5]:
splited_words = set()
for sentence in sentences:
    for word in sentence.split():
        splited_words.add(word)

# shuffle
vocab = {token: idx for idx, token in enumerate(splited_words)}
vocab['<pad>'] = len(vocab)
vocab['<sos>'] = len(vocab)
vocab['<eos>'] = len(vocab)
vocab['[MASK]'] = len(vocab)

inverse_vocab = {v: k for k, v in vocab.items()}
vocab_size = len(vocab)

def tokenize(sentence):
    return [vocab[word] for word in sentence.split()]

data = [tokenize(sentence) for sentence in sentences]

max_len = max(len(sentence) for sentence in data)
padded_data = [[vocab['<sos>']] + sentence + [vocab['<eos>']] + [vocab['<pad>']] * (max_len - len(sentence)) for sentence in data]
padded_data = np.array(padded_data)

## Modeling

In [6]:
def get_sinusoid_encoding_table(n_seq, d_hidn):
    # refs: https://paul-hyun.github.io/transformer-01/
    def cal_angle(position, i_hidn):
        return position / np.power(10000, 2 * (i_hidn // 2) / d_hidn)
    def get_posi_angle_vec(position):
        return [cal_angle(position, i_hidn) for i_hidn in range(d_hidn)]

    sinusoid_table = np.array([get_posi_angle_vec(i_seq) for i_seq in range(n_seq)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # even index sin 
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # odd index cos

    return sinusoid_table

class GPT(layers.Module):
    def __init__(self, vocab_size, embed_size, num_heads, num_layers, max_len=16):
        super().__init__()
        self.embed_size = embed_size
        self.embedding = layers.Embedding(vocab_size, embed_size)
        self.positional_encoding = np.expand_dims(get_sinusoid_encoding_table(max_len, embed_size), axis=0)
        self.transformer_decoder_layer = layers.TransformerDecoderLayer(
            d_model=embed_size,
            nhead=num_heads,
            dim_feedforward=embed_size * 4,
            batch_first=True
        )
        self.transformer_decoder = layers.TransformerDecoder(
            self.transformer_decoder_layer,
            num_layers=num_layers,
        )
        self.fc_out = layers.Linear(embed_size, vocab_size)

    def forward(self, tgt):
        tgt_len = tgt.shape[1]
        tgt_mask = self._generate_square_subsequent_mask(tgt_len)

        tgt_emb = self.embedding(tgt) + self.positional_encoding[:, :tgt_len, :]

        output = self.transformer_decoder(tgt_emb, tgt_emb, tgt_mask=tgt_mask)
        output = self.fc_out(output)

        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = np.triu(np.ones((sz, sz)), 1)
        return mask

class BERT(layers.Module):
    def __init__(self, vocab_size, embed_size, num_heads, num_layers, max_len=17):
        super().__init__()
        self.embed_size = embed_size
        self.embedding = layers.Embedding(vocab_size, embed_size)
        self.positional_encoding = np.expand_dims(get_sinusoid_encoding_table(max_len, embed_size), axis=0)
        self.transformer_encoder_layer = layers.TransformerEncoderLayer(
            d_model=embed_size,
            nhead=num_heads,
            dim_feedforward=embed_size * 4,
            batch_first=True
        )
        self.transformer_encoder = layers.TransformerEncoder(
            self.transformer_encoder_layer,
            num_layers=num_layers
        )
        self.fc_out = layers.Linear(embed_size, vocab_size)

    def forward(self, src):
        src_len = src.shape[1]
        src_emb = self.embedding(src) + self.positional_encoding[:, :src_len, :]
        
        output = self.transformer_encoder(src_emb)
        output = self.fc_out(output)
        
        return output

## Train

### GPT

In [7]:
embed_size = 128
num_heads = 8
num_layers = 2

gpt = GPT(vocab_size, embed_size, num_heads, num_layers)
criterion = layers.CrossEntropyLoss()
optimizer = optim.Adam(gpt, lr_rate=1e-3)

inputs = padded_data[:, :-1]
targets = padded_data[:, 1:]
bsz, seq_len = inputs.shape

num_epochs = 100
pbar = tqdm(range(num_epochs))
for epoch in pbar:
    outputs = gpt(inputs)
    
    outputs = outputs.reshape(-1, vocab_size)
    targets = targets.reshape(-1)
    loss = criterion(outputs, targets)

    dz = criterion.backward()
    dz = dz.reshape(bsz, seq_len, -1)
    optimizer.update(dz)
    
    pbar.set_description(f'Loss: {loss.item():5f}')

  0%|          | 0/100 [00:00<?, ?it/s]

Loss: 0.040975: 100%|██████████| 100/100 [01:43<00:00,  1.04s/it]


### BERT

In [8]:
import copy

embed_size = 128
num_heads = 8
num_layers = 2

bert = BERT(vocab_size, embed_size, num_heads, num_layers)
criterion = layers.CrossEntropyLoss()
optimizer = optim.Adam(bert, lr_rate=1e-3)

targets = padded_data
bsz, seq_len = targets.shape

num_epochs = 100
pbar = tqdm(range(num_epochs))
for epoch in pbar:
    inputs = copy.deepcopy(padded_data)

    # random masking
    # masking ratio: 30%
    for i in range(len(inputs)):
        sentence = inputs[i]
        masking_ids = np.random.randint(1,len(sentence)-1,size=int(len(sentence)*0.3))
        for idx in masking_ids:
            inputs[i][idx] = vocab['[MASK]']
    
    outputs = bert(inputs)
    
    outputs = outputs.reshape(-1, vocab_size)
    targets = targets.reshape(-1)
    loss = criterion(outputs, targets)

    dz = criterion.backward()
    dz = dz.reshape(bsz, seq_len, -1)
    optimizer.update(dz)
    
    pbar.set_description(f'Loss: {loss.item():5f}')

Loss: 0.487052: 100%|██████████| 100/100 [01:03<00:00,  1.57it/s]


## Predict

### GPT

In [11]:
def generate_sentence(model, start_sentence, max_len):
    generated = [vocab[token] for token in start_sentence.split()]
    input_seq = np.expand_dims(np.array(generated),0)
    
    while len(input_seq[0]) < max_len:
        # print(' '.join(inverse_vocab[token] for token in input_seq[0]))
        output = model.forward(input_seq)
        next_token_logits = output[-1, -1]
        next_token = np.argmax(next_token_logits).item()
        generated.append(next_token)
        if next_token == vocab['<eos>']:
            break
        input_seq = np.array([generated])
        
    return ' '.join(inverse_vocab[token] for token in generated)

sentence = "Great minds think alike, but they also think differently"
input_sentence = "Great minds think alike, but"
print(f"original: {sentence}")
print(f'input_sentence: {input_sentence}')
print(f'greedy: {generate_sentence(gpt, input_sentence, max_len=16)}')

original: Great minds think alike, but they also think differently
input_sentence: Great minds think alike, but
greedy: Great minds think alike, but they also be open to feedback Great minds think alike, but


### BERT

In [10]:
from hcrot.utils import softmax

def bert_predict_mask_token(model, sentence):
    inputs = tokenize(sentence)
    inputs = [[vocab['<sos>']] + inputs + [vocab['<eos>']]]
    mask_idx = inputs[0].index(vocab['[MASK]'])

    inputs = np.array(inputs)
    output = model.forward(inputs)

    logits = softmax(output[0][mask_idx])
    predict = sorted([(inverse_vocab[token],round(logit,2)) for token, logit in enumerate(logits)],key=lambda e: -e[1])[:5]
    
    output = output.argmax(-1)[0]
    return ' '.join(inverse_vocab[token] for token in output), predict

sentence = "Great minds think alike, but they also think differently"
masked_sentence = "Great minds [MASK] alike, but they also think differently"
prediction, token_logits = bert_predict_mask_token(bert, masked_sentence)
print(f"original: {sentence}")
print(f"predict: {prediction}")
print(f"token(prob): {' | '.join([f'{token}({logit})' for token, logit in token_logits])}")

original: Great minds think alike, but they also think differently
predict: <sos> Great minds think alike, but they also think differently <eos>
token(prob): think(0.34) | alike,(0.18) | they(0.05) | is(0.02) | Honesty(0.01)
